In [1]:
!pip install torch pandas numpy matplotlib scikit-learn
!pip install transformers sentence-transformers kobert-transformers
!pip install umap-learn hdbscan bertopic
!pip install kss

import torch
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
import plotly.express as px
import plotly.io as pio
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import kss
#---------------------- 리뷰 크롤링 ------------------------
import pandas as pd
from requests import request, get
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import time
import base64
from urllib.parse import quote, unquote, urlencode
import json
import csv
import requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
headers = {
            "Content-Type": "application/json",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "*/*",
            "Referer": "https://www.airbnb.co.kr/",
            "Accept-Language": "ko-KR,ko;q=0.9",
            "x-airbnb-api-key": "d306zoyjsyarp7ifhu67rjxn52tv0t20",
        }

# 번역본 없으면 걍 원본 ㄱㄱ가 아니라 걍 없애
def comments(r):
    if r["language"] == 'ko':
        return r["comments"]
    else:
        try:
            return r["localizedReview"]["comments"]
        except:
            return None

# 얘는 리뷰가져올 json임
def getReviewsJson(stay_id, limit, offset, headers=headers):
    jurl = "https://www.airbnb.co.kr/api/v3/StaysPdpReviewsQuery/dec1c8061483e78373602047450322fd474e79ba9afa8d3dbbc27f504030f91d?"
    variables = {
        "id":stay_id,
        "pdpReviewsRequest":{
            "fieldSelector":"for_p3_translation_only",
            "forPreview":False,
            "limit":limit,
            "offset":str(offset),
            "showingTranslationButton":False,
            "first":limit,
            "sortingPreference":"MOST_RECENT",
            "checkinDate":"2025-06-27",
            "checkoutDate":"2025-07-02",
            "numberOfAdults":"1",
            "numberOfChildren":"0",
            "numberOfInfants":"0",
            "numberOfPets":"0",
            "after":None
        }
    }

    extensions = {
        "persistedQuery":{
            "version":1,
            "sha256Hash":"dec1c8061483e78373602047450322fd474e79ba9afa8d3dbbc27f504030f91d"
        }
    }

    params = {
        "operationName":"StaysPdpReviewsQuery",
        "locale":"ko",
        "currency":"KRW",
        "variables":json.dumps(variables, separators=(',',':')),
        "extensions":json.dumps(extensions, separators=(',',':'))
    }

    resp = requests.get(jurl+urlencode(params, quote_via=quote), headers=headers)
    return resp.json()

def clean_text(text):
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = text.replace('<br>', '').replace('<br/>', '').replace('<br />', '')
    text = re.sub(r'[^\w\s.,$!?가-힣]|[/\\[<a-z>]:\[\]{}|]', '', text)
    text = re.sub(r'[!\"“”‘’?.]+', '. ', text)
    text = re.sub(r'[():]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

In [4]:
## 여기서 url만 가져오면 됨!!!
url = 'https://www.airbnb.co.kr/rooms/848425589512158141?check_in=2025-06-20&check_out=2025-06-22&photo_id=1611822771&source_impression_id=p3_1747354714_P3xYNIXjRSllUNzN&previous_page_section_name=1000'

# ---------- 여기서 url만 가져오면 됨!!! -------------


## 여기서부터는 안건드려도 됨
resp = request('get',url)
dom = BeautifulSoup(resp.text, 'html.parser')

p1, p2 = re.split(r'[?]', url)
review = p1+'/reviews?'+p2 # 리뷰 페이지 가는거

url, num = re.split(r'/rooms/', p1) # num은 숙소번호
num = num.split('/reviews')[0] # 진짜 숙소번호
encoding = 'StayListing:'+num # 인코딩 하려고
stay_id = base64.b64encode(encoding.encode('utf-8')).decode('utf-8')

limit, cnt = 50, 50
offset = 0
data = []
while 1:
    res_json = getReviewsJson(stay_id, limit, offset)
    try:
        review_data = res_json["data"]["presentation"]["stayProductDetailPage"]["reviews"]
        reviewsCount = int(review_data['metadata']['reviewsCount'])
        for r in review_data["reviews"]:
            comment_text = comments(r)
            if comment_text:
                data.append({
                    "user_id": r["reviewer"]["id"],
                    "name": r["reviewer"]["firstName"],
                    "stay_id": num,
                    "comment": comment_text,
                    "rating": r.get("rating"),
                    "createdAt": r.get("createdAt")
                })


    except KeyError:
        break

    if offset >= reviewsCount:
        break

    offset += cnt
    limit += cnt

reviews = pd.DataFrame(data)
reviews = reviews.dropna()
reviews.info()

#---------------------- 모델 사용 ------------------------

rows = []
reviews = reviews.reset_index(drop=True)
for j, row in reviews.iterrows():
    raw_comment = row['comment']
    user_id = row['user_id']
    cleaned_comment = clean_text(raw_comment)
    sentences = kss.split_sentences(cleaned_comment)

    idx = 0
    split_num = 1

    for sentence in sentences:
        sentence = sentence.strip()
        split_sentences = re.split(r'(?<=[가-힣\w])\.(?=[^\d])', sentence)
        if not isinstance(split_sentences, list):
            split_sentences = [sentence]

        for s in split_sentences:
            s = s.strip()
            if len(s) < 7:
                continue
            start = cleaned_comment.find(s, idx)
            end = start + len(s) - 1
            idx = end + 1
            rows.append({
                "stay_id": num,
                "user_id": user_id,
                "splitNum": split_num,
                "sentence": s,
                "start": start,
                "end": end
            })
            split_num += 1

# DataFrame 생성
sentence = pd.DataFrame(rows, columns=["stay_id", "user_id", "splitNum", "sentence", "start", "end"])
sentence.info()
# Plotly 렌더링 설정
pio.renderers.default = 'colab'

sentence = sentence['sentence']
docs = sentence.dropna().drop_duplicates().tolist()  # NaN 및 중복 제거
if len(docs) < 5:
    raise ValueError("데이터가 충분하지 않습니다.")

# MPS 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 한국어 Sentence-BERT 임베딩 모델 로드 (더 강력한 모델 사용)
embedding_model = SentenceTransformer("jhgan/ko-sbert-nli", device=device)

# 사용자 정의 seed topics 및 이름 (더 구체적이고 데이터 반영)
seed_topics = [
    ["청결도", "깨끗함", "깨끗", "더럽", "드러움", "드럽", "개더럽", "개더러움", "개드럽", "개드러움", "위생", "청소", "더러움", "청결", "불결", "정리", "오염", "깔끔함", "먼지", "청소상태", "냄새", "악취", "향기", "쾌쾌함", "냄새남", "냄새나", "향", "지린내", "곰팡이냄새", "청국장냄새", "냄새문제", "상쾌함", "환기"],
    ["위치", "교통", "가까움", "편리함", "접근성", "원거리", "교통편", "중앙", "외진", "이동", "전망", "근처"],
    ["가격", "비쌈", "저렴함", "가성비", "비용", "고가", "합리적", "경제적", "비싸", "저렴", "요금", "가치", "지불"],
    ["시설", "편리함", "새거", "새것", "새" "편함", "구비", "편의", "시설물", "장비", "부족", "완비", "기능", "설비", "낡음", "오래됨"],
    ["호스트", "서비스", "친절함", "응대", "도움", "불친절", "서비스품질", "관리", "지원", "배려", "체크인", "체크아웃", "입실", "퇴실", "환영", "지연", "빠름", "수속", "접수", "퇴소", "안내"],
    ["소음", "조용함", "시끄러움", "시끄럼", "방음", "소음문제", "고요", "고요함", "소란", "방음효과", "조용", "잡음", "소음원", "방해"],
]
topic_names = ["청결도", "위치", "가격", "시설", "호스트", "소음"]

# UMAP + HDBSCAN 설정 (노이즈 감소)
umap_model = UMAP(n_components=2, random_state=42, metric='cosine', n_neighbors=30, min_dist=0.1)
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=6, min_samples=3, cluster_selection_method='leaf')

# BERTopic 모델 생성 및 학습
topic_model = BERTopic(
    embedding_model=embedding_model,
    language="multilingual",
    min_topic_size=6,
    nr_topics="auto",
    seed_topic_list=seed_topics,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True,
    vectorizer_model=TfidfVectorizer()  # TF-IDF로 키워드 가중치 반영
)

topics, probs = topic_model.fit_transform(docs)

# 토픽 번호와 키워드 추출
topic_info = topic_model.get_topic_info()
topic_ids = topic_info[topic_info['Topic'] != -1]['Topic'].tolist()

# 토픽 임베딩 기반 유사도 계산 (더 정확한 매핑)
topic_embeddings = topic_model.topic_embeddings_[1:]  # 노이즈 제외
seed_embeddings = embedding_model.encode([" ".join(keywords) for keywords in seed_topics])

similarity = cosine_similarity(topic_embeddings, seed_embeddings)

# 각 토픽을 가장 유사한 seed topic으로 매핑
mapped_topics = {}
for i, sim_row in enumerate(similarity):
    best_idx = sim_row.argmax()
    mapped_topics[topic_ids[i]] = topic_names[best_idx]

selected_labels = topic_names
selected_topic_nums = [k for k, v in mapped_topics.items() if v in selected_labels]
filtered_indices = [i for i, t in enumerate(topics) if t in selected_topic_nums]
filtered_docs = [docs[i] for i in filtered_indices]

# 임베딩 및 UMAP 축소
filtered_embeddings = embedding_model.encode(filtered_docs)
filtered_reduced = umap_model.fit_transform(filtered_embeddings)

# 토픽 이름 기준 문장 분류 (기타 제외)
topic_sentences = defaultdict(list)
for doc, topic_id in zip(docs, topics):
    label = mapped_topics.get(topic_id, "기타")
    if label != "기타":
        topic_sentences[label].append(doc)

# 보기 좋게 출력
for topic_name in topic_names:
    topics = topic_sentences.get(topic_name, [])
    if not topics:
        continue
    print(f"\n토픽: {topic_name}(리뷰 갯수: {len(topics)})")
    for i, topic in enumerate(topics[:5]):  # 최대 5개만 미리보기 출력
        print(f"  {i+1}. {topic}")
    if len(topic) > 5:
        print(f"  ... (총 {len(topics)}개 리뷰 중 5개만 표시)")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    119 non-null    object
 1   name       119 non-null    object
 2   stay_id    119 non-null    object
 3   comment    119 non-null    object
 4   rating     119 non-null    int64 
 5   createdAt  119 non-null    object
dtypes: int64(1), object(5)
memory usage: 5.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   stay_id   347 non-null    object
 1   user_id   347 non-null    object
 2   splitNum  347 non-null    int64 
 3   sentence  347 non-null    object
 4   start     347 non-null    int64 
 5   end       347 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 16.4+ KB
Using device: cuda


2025-05-16 00:19:02,630 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2025-05-16 00:19:03,350 - BERTopic - Embedding - Completed ✓
2025-05-16 00:19:03,358 - BERTopic - Guided - Find embeddings highly related to seeded topics.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-05-16 00:19:03,472 - BERTopic - Guided - Completed ✓
2025-05-16 00:19:03,474 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-16 00:19:05,817 - BERTopic - Dimensionality - Completed ✓
2025-05-16 00:19:05,818 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-16 00:19:05,827 - BERTopic - Cluster - Completed ✓
2025-05-16 00:19:05,828 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-05-16 00:19:05,862 - BERTopic - Representation - Completed ✓
2025-05-16 00:19:05,863 - BERTopic - Topic reduction - Reducing number of topics
2025-05-16 00:19:05,874 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-16 00:19:05,892 - BERTopic - Representation - Completed ✓
2025-05-16 00:19:05,894 - BERTopic - Topic reduction - Reduced number of topics from 20 to 6



토픽: 청결도(리뷰 갯수: 6)
  1. 어두운 편이라 화장 할 때는 불편할 것 같아요.
  2. 겨울이라 그런지 아님 먼지가 약간 있어서 그런지 조금 건조했던 것 빼고는 불편한 점 하나 없이 편안하게 묵었어요.
  3. 다만 침구류가 호텔식이라 먼지가 좀 날리는 것이 아쉬웠던 것 같아용
  4. 따뜻한물도 잘나옴근데 샤워기가 자유분방해서 방심하면 코브라댄스춤욕실이 좁은건 아닌데 좀 신기해서 방심하면 휴지 다젖음수건 넉넉히 주는데 냄새가 쪼끔 남방음 잘되는듯 영화틀고 깔깔웃어도 누구 안찾아옴엘베가 종종 말썽이긴 했는데 담배냄새 안나고 술냄새나 음식물냄새 안나는건 좋앗
  5. 뜨거운물 수압이 많이 낮았는데 수도를 거슬러 올라가서 밸브를 좀 여니까 괜찮더라구요.
  ... (총 6개 리뷰 중 5개만 표시)

토픽: 위치(리뷰 갯수: 214)
  1. 숙소 깨끗하고 탁트인 오션뷰가 좋네요
  2. 출발 전부터 친절하고 꼼꼼하게 챙겨주셔서 편하게 지낼수 있었어요
  3. 우리의 우려 사항을 해결하기 위해 매우 적극적으로 도움을 주었고, 사진 안내가 체크인에 확실히 도움이 되었습니다.
  4. 보안이 있어서 안전하다고 느꼈고, 그냥 비밀번호를 알고 있다고 알렸습니다.
  5. 밤에 드론쇼와 주말에 공연이 열리는 인기 명소이기 때문에 이 지역 자체가 붐비기도 합니다.
  ... (총 214개 리뷰 중 5개만 표시)

토픽: 소음(리뷰 갯수: 6)
  1. 단점은 도로 옆에 있어 밤에 조금 시끄러우며 방이 작다는 것입니다.
  2. 밤에는 거리에서 음악이 꽤 시끄럽지만, 방의 방음은 소음이 들리지 않을 정도로 아주 좋습니다.
  3. 밤에 버스킹 공연이나 그런 소음은 심한편임.
  4. 영화틀면 무시가능.
  5. 에어컨이 시끄러워서 가을이나 겨울에 오면 더 좋을것 같다는 생각이 들었어요.
  ... (총 6개 리뷰 중 5개만 표시)
